In [49]:
import pandas as pd

file_to_load = "../airbnb/AirBnB 2019/calendar_2019.csv"
list_to_load = "../airbnb/AirBnB 2019/listings_2019.csv"

cal_2019 = pd.read_csv(file_to_load)
list_2019 = pd.read_csv(list_to_load, low_memory = False)
list_2019_price = list_2019[['id','price']]
list_2019_price = list_2019_price.rename(columns = {'id':'listing_id'})
list_2019_price

,listing_id,price
0,9835,60
1,10803,35
2,12936,159
3,15246,50
4,16760,69
...,...,...
22890,30592332,55
22891,30592345,129
22892,30592362,53
22893,30592590,130


In [63]:
sum_list_2019 = list_2019[['id','name','host_response_time','host_response_rate','host_is_superhost',
                       'street','neighborhood','city','suburb','state','zipcode',
                       'country','latitude','longitude','property_type','room_type',
                       'accommodates','bathrooms','bedrooms','beds', 'amenities','price',
                       'security_deposit', 'cleaning_fee', 'has_availability',
                       'availability_365', 'number_of_reviews','review_scores_rating',
                       'review_scores_accuracy', 'review_scores_cleanliness',
                       'review_scores_checkin', 'review_scores_communication',
                       'review_scores_location', 'review_scores_value','reviews_per_month']]
df = sum_list_2019.loc[sum_list_2019['neighborhood'].isna() == True]
df

,id,name,host_response_time,host_response_rate,host_is_superhost,street,neighborhood,city,suburb,state,...,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
5,38271,Melbourne - Old Trafford Apartment,within an hour,100%,t,"Berwick, VIC, Australia",NaN,Casey,Berwick,VIC,...,328,112,97.0,10.0,10.0,10.0,10.0,10.0,10.0,1.14
6,41836,CLOSE TO CITY & MELBOURNE AIRPORT,NaN,NaN,f,"Reservoir, VIC, Australia",NaN,Darebin,Reservoir,VIC,...,0,159,94.0,9.0,9.0,10.0,10.0,9.0,9.0,1.62
7,43414,Home In The City,within an hour,100%,f,"East Melbourne, VIC, Australia",NaN,Melbourne,East Melbourne,VIC,...,11,3,93.0,10.0,9.0,9.0,10.0,9.0,9.0,0.29
8,43429,Tranquil Javanese-Style Apartment in Oakleigh ...,within an hour,100%,t,"Oakleigh East, VIC, Australia",NaN,Monash,Oakleigh East,VIC,...,310,168,97.0,10.0,10.0,10.0,10.0,10.0,10.0,1.72
9,44082,Queen Room in Beautiful House,within an hour,100%,t,"Frankston, VIC, Australia",NaN,Frankston,Frankston,VIC,...,310,28,99.0,10.0,10.0,10.0,10.0,10.0,10.0,0.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22881,30590430,1x Bedroom very close to Trains Lakes Shops,NaN,NaN,f,"Pakenham, VIC, Australia",NaN,Cardinia,Pakenham,VIC,...,56,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22885,30591274,Melbourne shared triple bunk room,NaN,NaN,f,"Craigieburn, VIC, Australia",NaN,Hume,Craigieburn,VIC,...,164,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22886,30591343,1 Bedroom very close to Trains Shops & Lakes,NaN,NaN,f,"Pakenham, VIC, Australia",NaN,Cardinia,Pakenham,VIC,...,56,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22891,30592345,Melton South Getaway,within an hour,100%,f,"Melton South, VIC, Australia",NaN,Melton,Melton South,VIC,...,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Which neighbourhoods have the most individual listings?

## How has this changed between datasets

## Which neighbourhoods are the most expensive?

## What price variance do we see in a suburb and why?

## 

In [47]:
file_to_load = "../airbnb/AirBnB 2020/calendar_2020.csv"
cal_2020 = pd.read_csv(file_to_load)

cal_2020['date'] = pd.to_datetime(cal_2020['date'])
cal_2020['day'] = cal_2020['date'].dt.day
cal_2020['month'] = cal_2020['date'].dt.month
cal_2020['year'] = cal_2020['date'].dt.year

month_2020 = cal_2020.groupby(['year','month']).count()
month_2020

listing_id    date  available   price  adjusted_price  \
year month                                                          
2020 8          233985  233985     233985  233985          233985   
     9          612600  612600     612600  612600          612600   
     10         633020  633020     633020  633020          633020   
     11         612600  612600     612600  612600          612600   
     12         633020  633020     633020  633020          633020   
2021 1          633020  633020     633020  633020          633020   
     2          571760  571760     571760  571760          571760   
     3          633020  633020     633020  633020          633020   
     4          612600  612600     612600  612600          612600   
     5          633020  633020     633020  633020          633020   
     6          612600  612600     612600  612600          612600   
     7          633020  633020     633020  633020          633020   
     8          399415  399415     399415  399415          399415   

            minimum_nights  maximum_nights     day  
year month                                          
2020 8              233621          233621  233985  
     9              612600          612600  612600  
     10             633020          633020  633020  
     11             612600          612600  612600  
     12             633020          633020  633020  
2021 1              633020          633020  633020  
     2              571760          571760  571760  
     3              633020          633020  633020  
     4              612600          612600  612600  
     5              633020          633020  633020  
     6              612600          612600  612600  
     7              633020          633020  633020  
     8              399415          399415  399415

In [48]:
file_to_load = "../airbnb/AirBnB 2021/calendar_2021.csv"
cal_2021 = pd.read_csv(file_to_load)

cal_2021['date'] = pd.to_datetime(cal_2021['date'])
cal_2021['day'] = cal_2021['date'].dt.day
cal_2021['month'] = cal_2021['date'].dt.month
cal_2021['year'] = cal_2021['date'].dt.year

month_2021 = cal_2021.groupby(['year','month']).count()
month_2021

listing_id    date  available   price  adjusted_price  \
year month                                                          
2021 4          357813  357813     357813  357813          357813   
     5          582707  582707     582707  582707          582707   
     6          563910  563910     563910  563910          563910   
     7          582707  582707     582707  582707          582707   
     8          582707  582707     582707  582707          582707   
     9          563910  563910     563910  563910          563910   
     10         582707  582707     582707  582689          582689   
     11         563910  563910     563910  563880          563880   
     12         582707  582707     582707  582676          582676   
2022 1          582707  582707     582707  582676          582676   
     2          526316  526316     526316  526288          526288   
     3          582712  582712     582712  582681          582681   
     4          207477  207477     207477  207450          207450   

            minimum_nights  maximum_nights     day  
year month                                          
2021 4              356447          356447  357813  
     5              582707          582707  582707  
     6              563910          563910  563910  
     7              582707          582707  582707  
     8              582707          582707  582707  
     9              563910          563910  563910  
     10             582707          582707  582707  
     11             563910          563910  563910  
     12             582707          582707  582707  
2022 1              582707          582707  582707  
     2              526316          526316  526316  
     3              582712          582712  582712  
     4              207477          207477  207477

In [35]:
days_cost = pd.merge(cal_2019, list_2019_price, on = 'listing_id', how = 'left')
daily_price_2019 = days_cost.groupby('listing_id').mean()
daily_price_2019.describe()

,day,month,year,price_y
count,22895.000000,2.289500e+04,22895.000000,22895.000000
mean,15.720548,6.526027e+00,2018.931509,148.004368
std,0.000000,8.881978e-16,0.000070,210.883253
min,15.720548,6.526027e+00,2018.931507,0.000000
25%,15.720548,6.526027e+00,2018.931507,71.000000
50%,15.720548,6.526027e+00,2018.931507,111.000000
75%,15.720548,6.526027e+00,2018.931507,165.000000
max,15.720548,6.526027e+00,2018.934247,12624.000000


In [29]:
#Finding all available booking ID's and total days available
avail_2019 = cal_2019.loc[cal_2018['available'] == 't']
avail_2019 = avail_2019.groupby('listing_id').count()
avail_2019 = pd.merge(avail_2019, list_2019_price, on = 'listing_id', how = 'left')
avail_2019 = avail_2019.rename(columns = {'available':'Number of Days Available',
                                          'price_y': 'Price per Night'})
avail_2019 = avail_2019[['listing_id','Number of Days Available','Price per Night']]
avail_2019

,listing_id,Number of Days Available,Price per Night
0,9835,365,60
1,10803,213,35
2,12936,326,159
3,16760,328,69
4,38271,328,98
...,...,...,...
17392,30592332,82,55
17393,30592345,365,129
17394,30592362,15,53
17395,30592590,349,130


In [30]:
#Finding all booked listing_id's and number of days booked
booked_2019 = cal_2019.loc[cal_2019['available'] == 'f']
booked_2019 = booked_2019.groupby('listing_id').count()
booked_2019 = pd.merge(booked_2019, list_2019_price, on = 'listing_id', how = 'left')
booked_2019 = booked_2019.rename(columns = {'available':'Number of Days Booked',
                                          'price_y': 'Price per Night'})
booked_2019 = booked_2019[['listing_id','Number of Days Booked','Price per Night']]
booked_2019

,listing_id,Number of Days Booked,Price per Night
0,10803,152,35
1,12936,39,159
2,15246,365,50
3,16760,37,69
4,38271,37,98
...,...,...,...
22326,30592128,336,30
22327,30592332,283,55
22328,30592362,350,53
22329,30592590,16,130


In [31]:
cal_group = days_cost.groupby(['listing_id','available']).count()

cal_group

date  price_x  price_y
listing_id available                        
9835       t           365      365      365
10803      f           152        0      152
           t           213      213      213
12936      f            39        0       39
           t           326      326      326
...                    ...      ...      ...
30592362   t            15       15       15
30592590   f            16        0       16
           t           349      349      349
30593623   f           297        0      297
           t            68       68       68

[39728 rows x 3 columns]

In [32]:
cal_2019['date'] = pd.to_datetime(cal_2019['date'])
cal_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8356675 entries, 0 to 8356674
Data columns (total 4 columns):
 #   Column      Dtype         
---  ------      -----         
 0   listing_id  int64         
 1   date        datetime64[ns]
 2   available   object        
 3   price       object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 255.0+ MB


In [33]:
cal_2019['day'] = cal_2019['date'].dt.day
cal_2019['month'] = cal_2019['date'].dt.month
cal_2019['year'] = cal_2019['date'].dt.year

In [34]:
month_2019 = cal_2019.groupby(['year','month']).count()
month_2019

listing_id    date  available   price     day
year month                                               
2018 12         572360  572360     572360  186899  572360
2019 1          709745  709745     709745  312593  709745
     2          641060  641060     641060  323736  641060
     3          709745  709745     709745  307871  709745
     4          686850  686850     686850  295074  686850
     5          709745  709745     709745  315288  709745
     6          686850  686850     686850  231450  686850
     7          709745  709745     709745  226422  709745
     8          709745  709745     709745  227130  709745
     9          686850  686850     686850  207415  686850
     10         709745  709745     709745  213627  709745
     11         686850  686850     686850  205241  686850
     12         137385  137385     137385   41446  137385